z# 📘 Dokumentacja Techniczna Solvera: Analiza Przekroju Cienkościennego

## 1. Wstęp i Cel Analizy
Niniejszy dokument stanowi szczegółowy opis architektury i logiki działania modułu obliczeniowego zawartego w pliku `solver.py`. Narzędzie to służy do zaawansowanej analizy wytrzymałościowej i statecznościowej **asymetrycznego słupa złożonego**, składającego się z gorącowalcowanego ceownika (np. UPE/UPN) wzmocnionego dospawanym płaskownikiem.

Ze względu na brak podwójnej symetrii przekroju, analiza wykracza poza elementarną wytrzymałość materiałów, uwzględniając:
* **Teorię Własowa** dla prętów cienkościennych o przekroju otwartym (zjawisko spaczenia i bimomentu).
* **Wyboczenie giętno-skrętne** (sprzężenie formy giętnej i skrętnej).
* Wytyczne normy **PN-EN 1993-1-1 (Eurokod 3)** w zakresie klasyfikacji przekroju i interakcji sił.

Solver został zaprojektowany jako **funkcja bezstanowa** (`pure function`) o nazwie `analizuj_przekroj_pelna_dokladnosc`, która przyjmuje komplet danych wejściowych i zwraca deterministyczny raport wyników.

---

## 2. Architektura Danych Wejściowych

Funkcja solera przyjmuje cztery słowniki konfiguracyjne. Taka separacja danych od logiki pozwala na łatwą parametryzację, optymalizację i testowanie różnych scenariuszy (np. zmiana materiału ze stali na aluminium).

### Struktury Danych (Słowniki):

1.  **`upe_data`** (Katalog Ceowników):
    Zawiera geometrię profilu bazowego pobraną z bazy `katalog.py`.
    * Klucze: `hc` (wysokość), `bc` (szerokość), `twc`/`tfc` (grubości ścianek), `Ac` (pole), `Icy`/`Icz` (momenty bezwładności własne), `xc` (położenie $S_c$ lokalnego).

2.  **`geo_data`** (Geometria Płaskownika):
    Parametry elementu wzmacniającego, które determinują asymetrię układu.
    * Klucze: `bp` (szerokość płaskownika), `tp` (grubość płaskownika).

3.  **`load_data`** (Obciążenia i Materiał):
    * **Siły:** $F_x$ (siła osiowa), $F_{promien}$ (mimośród przyłożenia siły względem krawędzi), $w_{Ty}, w_{Tz}$ (współczynniki sił tnących).
    * **Materiał:** $E$ (Moduł Younga), $G$ (Moduł Kirchhoffa), $R_e$ (Granica plastyczności), $E_{def}$ (parametr odniesienia dla klasyfikacji).

4.  **`safety_data`** (Bezpieczeństwo):
    Parametry sterujące konserwatyzmem obliczeń wg Eurokodu.
    * $\gamma_{M0}, \gamma_{M1}$ – częściowe współczynniki bezpieczeństwa.
    * $\alpha_{imp}$ – parametr imperfekcji geometrycznej (zależny od krzywej wyboczeniowej, np. 0.49 dla krzywej "c").

---

## 3. Krok A: Geometria Podstawowa i Twierdzenie Steinera

W pierwszej fazie kod oblicza "klasyczne" charakterystyki geometryczne całego przekroju złożonego. Ponieważ układ składa się z dwóch przesuniętych względem siebie figur (ceownik + płaskownik), konieczne jest wyznaczenie nowego, globalnego środka ciężkości.

### Logika Obliczeniowa
Kod wykorzystuje podstawowe operacje arytmetyczne oraz bibliotekę `numpy`.

1.  **Pola powierzchni:**
    $$A_{cal} = 2 \cdot A_c + (b_p \cdot t_p)$$

2.  **Środek Ciężkości ($y_c$):**
    Obliczony jako średnia ważona momentów statycznych pól składowych względem górnej krawędzi (osi płaskownika).
    $$y_c = \frac{\sum A_i \cdot y_i}{A_{cal}}$$

3.  **Momenty Bezwładności ($I_y, I_z$):**
    Zastosowanie **Twierdzenia Steinera**. Przekrój posiada jedną oś symetrii ($Y$), więc osie główne pokrywają się z osiami geometrycznymi.
    $$I_{złożony} = \sum \left( I_{własny} + A_i \cdot d_i^2 \right)$$
    Gdzie $d_i$ to odległość między środkiem ciężkości elementu składowego a nowym $S_c$.

---

## 4. Krok B: Klasyfikacja Przekroju (Eurokod 3)

Przed przystąpieniem do analizy naprężeń, algorytm musi określić **klasę przekroju** (1, 2, 3 lub 4). Jest to krok krytyczny, ponieważ:
* **Klasy 1-2:** Pozwalają na wykorzystanie plastycznej rezerwy nośności.
* **Klasa 3:** Ogranicza nośność do granicy sprężystości ($R_e$).
* **Klasa 4:** Wymaga uwzględnienia niestateczności miejscowej ścianek (przekrój efektywny).

### Algorytm decyzyjny (`if-elif-else`)
Solver analizuje osobno smukłość środnika i stopki:
1.  Oblicza parametr materiałowy $\epsilon = \sqrt{235/R_e}$.
2.  **Dla środnika:** Wyznacza strefę ściskaną ($\alpha_{web}$) na podstawie położenia osi obojętnej $y_c$.
3.  **Dla stopki:** Stopka jest elementem wspornikowym ściskanym równomiernie.
4.  **Porównanie:** Smukłość $\lambda = c/t$ jest porównywana z limitami normowymi (np. $33\epsilon, 38\epsilon$).

```python
# Fragment logiki klasyfikacji w solverze
if smuklosc_web <= limit_web_c1: klasa_web = 1
elif smuklosc_web <= limit_web_c2: klasa_web = 2
else: ...
klasa_przekroju = max(klasa_web, klasa_flange)

## 5. Krok C: Teoria Własowa – Analiza Sektorowa

Jest to "serce" solvera. Ponieważ przekrój jest monosymetryczny (posiada tylko oś $Y$), Środek Ścinania ($S_s$) nie pokrywa się ze Środkiem Ciężkości ($S_c$). Powoduje to powstanie mimośrodu $y_0 = \Delta y_s$, który sprzęga zginanie ze skręcaniem. 

Aby wyznaczyć parametry sektorowe, kod wykorzystuje bibliotekę `sympy` do całkowania symbolicznego. Zapewnia to idealną precyzję matematyczną bez błędów dyskretyzacji (meshowania).

### Metoda 4 Stref (Integration Zones)
Kontur przekroju dzielony jest na 4 logiczne odcinki, po których porusza się zmienna całkująca `s_var`:
* **Strefa 1:** Płaskownik (od osi symetrii do krawędzi wewnętrznej ceownika).
* **Strefa 2:** Zakładka (strefa podwójnej grubości: płaskownik + stopka).
* **Strefa 3:** Środnik ceownika (odcinek pionowy).
* **Strefa 4:** Dolna stopka ceownika (odcinek poziomy, powrotny).

### Algorytm wyznaczania funkcji wycinkowej $\omega(s)$:
**Całkowanie łańcuchowe:** Dla każdej strefy $i$ funkcja startuje od wartości końcowej strefy $i-1$, co zapewnia ciągłość:

$$\omega(s) = \int_0^s h(t) \cdot dt + \omega_{start}$$

Gdzie $h(t)$ to ramię prostopadłe od bieguna $S_c$ do stycznej konturu.

### Wyznaczanie Środka Ścinania ($S_s$):
Obliczenie momentu statycznego wycinkowego i przesunięcia bieguna:

$$\Delta y_s = \frac{1}{I_y} \int_A \omega_{Sc} \cdot z \cdot dA$$

### Transformacja do układu głównego:
Przeliczenie funkcji wycinkowej względem nowego bieguna $S_s$:

$$\omega_{Ss}(s) = \omega_{Sc}(s) - \Delta y_s \cdot z_{glob}(s)$$

### Parametry Sztywnościowe
Na podstawie $\omega_{Ss}$ obliczane są kluczowe stałe:
* $I_\omega$ (**Sztywność wycinkowa**): Odporność na spaczenie (warping).
    $$I_\omega = \int_A \omega_{Ss}^2 \cdot dA$$
* $I_t$ (**Moment skręcania swobodnego**): Suma sztywności poszczególnych ścianek prostokątnych (wzór Saint-Venanta dla profili otwartych).

---

## 6. Krok D: Siły Wewnętrzne i Bimoment

Solver redukuje obciążenia zewnętrzne do sił przekrojowych działających w Środku Ścinania.

### Specyfika Bimomentu ($B_\omega$)
Dla pręta cienkościennego utwierdzonego jednostronnie (wspornik), zablokowanie swobodnej deplanacji w utwierdzeniu generuje Bimoment. Jest to samorównoważący się układ sił wzdłużnych. 

Kod modeluje to zjawisko analitycznie:

1.  Oblicza charakterystykę giętną pręta $k$:
    $$k = \sqrt{\frac{G I_t}{E I_\omega}}$$

2.  Wyznacza rozkład bimomentu od momentu skręcającego $M_s$:
    $$B_\omega(x) = \frac{M_s}{k} \tanh(k \cdot L)$$

Funkcja `tanh` modeluje zanikanie wpływu utwierdzenia w głąb belki ("brzegowy efekt strefowy").

---

## 7. Krok E: Stateczność (Wyboczenie Giętno-Skrętne)

Solver rozwiązuje problem wartości własnych, aby wyznaczyć siłę krytyczną, przy której słup utraci stabilność. 

### Siły krytyczne Eulera:
Obliczane niezależnie dla form prostych:
* $N_{cr,y}$ (giętne względem osi słabej),
* $N_{cr,z}$ (giętne względem osi mocnej),
* $N_{cr,T}$ (czyste wyboczenie skrętne).

### Interakcja (Równanie Timoszenki):
Ze względu na asymetrię ($y_0 = \Delta y_s \neq 0$), formy giętne i skrętne są sprzężone. Kod rozwiązuje równanie kwadratowe dla wyboczenia giętno-skrętnego:

$$i_0^2 (N - N_z)(N - N_T) - N^2 y_0^2 = 0$$

Solver wybiera mniejszy z pierwiastków ($N_{cr,gs}$) jako miarodajną siłę krytyczną.

### Współczynniki redukcyjne $\chi$:
Przejście z teorii idealnej (Euler) na rzeczywistą (Eurokod). Obliczenie smukłości względnej $\bar{\lambda}$ i odczytanie współczynnika $\chi$ z odpowiedniej krzywej wyboczeniowej (uwzględnienie imperfekcji).

---

## 8. Krok F: Analiza Punktowa (Pętla Naprężeń)

Zamiast sprawdzać wytężenie w jednym punkcie, solver iteruje po zdefiniowanej liście Punktów Krytycznych (`punkty_def`), co pozwala na wykrycie lokalnych spiętrzeń naprężeń.

### Lista Punktów Kontrolnych:
* **P1:** Środek płaskownika (oś symetrii).
* **P2:** Koniec nakładki (skokowa zmiana sztywności).
* **P3:** Górne naroże ceownika.
* **P4:** Środek środnika (oś obojętna dla zginania $M_z$).
* **P5:** Dolne naroże.
* **P6:** Koniec dolnej stopki (Miejsce maksymalnego spaczenia $\omega_{max}$ – krytyczne dla Bimomentu).

### Obliczenia w pętli (dla każdego punktu):
1.  **Ewaluacja funkcji:** Podstawienie współrzędnych punktu do wzorów symbolicznych `sympy`.
2.  **Superpozycja Naprężeń Normalnych ($\sigma_{x}$):**
    $$\sigma_{total} = \frac{N}{A} + \frac{M_y}{I_y}z + \frac{M_z}{I_z}y + \frac{B_\omega}{I_\omega}\omega$$
3.  **Superpozycja Naprężeń Stycznych ($\tau$):**
    $$\tau_{total} = \tau_{Vy} + \tau_{Vz} + \tau_{skręcanie}$$
    (Wykorzystanie wzoru Żurawskiego $\frac{VS}{It}$ oraz liniowego rozkładu skręcania).
4.  **Hipoteza Wytężeniowa:**
    Obliczenie naprężenia zredukowanego wg HMH (Huber-Mises-Hencky):
    $$\sigma_{VM} = \sqrt{\sigma_{total}^2 + 3\tau_{total}^2}$$

---

## 9. Krok G: Finalna Weryfikacja (UR - Utilization Ratio)

Na końcu funkcja agreguje wszystkie wyniki w jeden globalny wskaźnik bezpieczeństwa, stosując inżynierską zasadę sumowania wytężeń (interakcja liniowa):

$$UR = \frac{|N_{Ed}|}{N_{Rd,stab}} + \frac{|M_{y,Ed}|}{M_{y,Rd}} + \frac{|M_{z,Ed}|}{M_{z,Rd,stab}} + \frac{|B_{Ed}|}{M_{\omega,Rd}}$$

### Interpretacja wyniku:
Mianowniki to nośności obliczeniowe zredukowane o współczynniki wyboczeniowe ($\chi$) i zwichrzeniowe ($\chi_{LT}$).
* **UR $\le$ 1.0:** Konstrukcja spełnia wymagania normowe.
* **UR > 1.0:** Przekroczenie stanu granicznego nośności (niebezpieczeństwo).

### Cechy Programistyczne
* **Symbolic Math:** Wykorzystanie `sympy` eliminuje błędy aproksymacji geometrii.
* **Zabezpieczenia:** Funkcja zawiera "guard clauses" (np. `max(0.001, ...)`), chroniące przed dzieleniem przez zero dla geometrii zdegenerowanych.
* **Modularność:** Kod jest niezależny od konkretnych wartości materiałowych, pobierając je dynamicznie ze słowników wejściowych.